In [ ]:
%load_ext autoreload

In [ ]:
from pathlib import Path
import numpy as np
from skimage import io as skio
import matplotlib.pyplot as plt
%matplotlib notebook

from split_dataset import SplitDataset
from fimpy.pipeline.alignment import align_2p_volume
from fimpy.pipeline.roi_extraction import correlation_map
%autoreload

In [ ]:
master = Path(r"\\FUNES\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\2p\habenula\fixed")
master = Path(r"\\Funes\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\2p\rf\fixed")
master = Path(r"\\FUNES\Shared\experiments\E0044_spontaneous\gad1b\2p_anatomy\gcamp6s")
#master = Path(r"\\Funes\Shared\experiments\E0044_spontaneous\itpr1b\cerebellum")
#master = Path(r"\\FUNES\Shared\experiments\E0020_random_coherences\v30_two_options_with_multiple_forward_tests_embedded_ls\2p\gad1b-6s\v33\new")

master = Path(r"\\Funes\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\2p\ipn\gad1b")
all_fish = list(master.glob("*f[0-9]*"))
fish_dir = all_fish[0]
print(fish_dir)

In [ ]:
source_dir = "anatomy\original"
source_dir = "original"

frames_per_plane = None
aligned_dir = "aligned"
force_realign = False
force_remake_tifs = False
force_remake_corr = False

In [ ]:
import flammkuchen as fl
import json
inc_factor = 2**12

metadata_file_stack = fish_dir / "original_o/stack_metadata.json"
with open(str(metadata_file_stack), "r") as fr:
    stack_param = json.load(fr)
n_z, nx, ny = stack_param["shape_full"][1:4]
stack = SplitDataset(fish_dir / "original_o/")
print(n_z)
for i in range(n_z):
    if i < 10:
        file_name = '000' + str(i) + '.h5'
    else:
        file_name = '00' + str(i) + '.h5'
    print(file_name)

    data_in = stack[:, i, :, :]
    print(np.max(data_in), np.min(data_in), np.mean(data_in))

    data_out = (data_in + 0.05) * inc_factor
    print(np.max(data_out), np.min(data_out), np.mean(data_out))
    print(np.percentile(data_in, 99.999))

    data_out = data_out.astype(np.uint16)
    data_out = np.expand_dims(data_out, 1)
    data_out_folder = str(fish_dir / "original" / file_name)

    fl.save(data_out_folder, {"stack_4D": data_out}, compression="blosc")
    data_in = 0
    data_out = 0


In [ ]:
#ds = SplitDataset(fish_dir / "original")
#stack = ds[:, :, :, :]
#print(np.shape(stack))

In [ ]:
align_2p_volume(SplitDataset(fish_dir / source_dir), output_dir=str(fish_dir), across_planes=False)

In [ ]:
ds = SplitDataset(fish_dir / "aligned")
stack = ds[:, :, :, :]

In [ ]:
print(stack.max(), stack.mean())
mean = stack.mean(0)
print(np.shape(mean))
skio.imsave(str(fish_dir / "anatomy.tif"), mean)

In [ ]:
ds = 0
stack = 0

In [ ]:
ds = SplitDataset(fish_dir / source_dir)
stack = ds[:, :, :, :]
print(stack.max(), stack.mean())
mean = stack.mean(0)
print(np.shape(mean))
skio.imsave(str(fish_dir / "anatomy_original.tif"), mean)

In [ ]:
mean = skio.imread(str(fish_dir /  "anatomy.tif"))

In [ ]:
align_2p_volume(SplitDataset(fish_dir / source_dir), output_dir=str(fish_dir), reference=mean, across_planes=False)
ds = SplitDataset(fish_dir / "aligned")
stack = ds[:, :, :, :]
print(stack.max(), stack.mean())
mean = stack.mean(0)
print(np.shape(mean))
skio.imsave(str(fish_dir / "anatomy2.tif"), mean.astype(np.uint16))
ds = 0
stack = 0

In [ ]:
#anatomy = skio.imread(str(fish_dir / "anatomy1.tif"))
#anatomy2 = anatomy.T
#print(np.shape(anatomy2))

In [ ]:
force_realign = True
force_remake_tifs = False 

for i in range(2, 5):
    
    anatomy_file = "anatomy" + str(i) + ".tif"
    
    # Create anatomy tif file:
    if not (fish_dir / anatomy_file).exists() or force_remake_tifs:
        print("Saving anatomy...")
        ds = SplitDataset(fish_dir / "aligned")
        stack = ds[:, :, :, :]
        print(stack.max())
        mean = stack.mean(0)

        skio.imsave(str(fish_dir / anatomy_file), mean.astype(np.uint16))
        stack = 0
        
    # Align:
    if not (fish_dir / aligned_dir).exists() or force_realign:
        print("Aligning...")

        anatomy = skio.imread(str(fish_dir / anatomy_file))
        #anatomy = anatomy.T
        align_2p_volume(SplitDataset(fish_dir / source_dir), output_dir=str(fish_dir), reference=anatomy, across_planes=False)
    else:
        print("Already aligned")

        

In [ ]:
if not (fish_dir / "corrmap.tif").exists():
    print("Saving correlation map")
    ds = SplitDataset(fish_dir / "aligned")
    corr_map = correlation_map(ds)[:, :, :]
    skio.imsave(str(fish_dir / "corrmap.tif"), corr_map.astype(np.float))
    
ds = 0

In [ ]:
import flammkuchen as fl
from fimpy.pipeline.roi_extraction import extract_traces

In [ ]:
# Extracting traces: 
# master = Path(r"C:\Users\lavian\Desktop\data\e0040_V13")
master = Path(r"\\Funes\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\2p\habenula\fixed")
master = Path(r"C:\Users\lavian\Desktop\data\e0040_V13\rf\fixed")
all_fish = list(master.glob("*f[0-9]*"))
fish_dir = all_fish[5]
print(fish_dir)

In [ ]:
dir_rois = str(fish_dir/'merged_rois.h5')
dir_aligned = str(fish_dir / "aligned")
aligned = SplitDataset(dir_aligned)

In [ ]:
rois_arr = fl.load(dir_rois)['stack'] 
rois_arr = rois_arr.astype(int) - 1
traces = extract_traces(aligned, rois_arr)
np.shape(rois_arr)

In [ ]:
print(dir_rois)